In [2]:
import pandas as pd
import numpy as np

# Data Cleaning & Manipulation
With Pandas, we can do more than just select data that is already there. 
We can add new columns to our datasets, apply functions, iterate thorugh each row in the dataframe, and more.

This is where we move from "pandas for exploring our data" to "pandas for getting our data ready to feed into models".

In [29]:
wine = pd.read_csv('data/wine_reviews/winemag-data_first150k.csv', index_col=0)
energy = pd.read_csv('data/energy/PJM_Load_hourly.csv', parse_dates=True, index_col=0)

### Dealing with Null Values:

In [3]:
print('Does the wine df conatin nulls?:', wine.isnull().any().any())
print('Does the energy df contain nulls?:', energy.isnull().any().any())

Does the wine df conatin nulls?: True
Does the energy df contain nulls?: False


**Question:** why did we use `.any()` twice when asking this question?

What happens if we just use `.any()` once?

In [5]:
# energy.info()
# wine.info()
wine.isnull().all()
wine.isnull().any()
wine.isnull().any().any()

country         True
description    False
designation     True
points         False
price           True
province        True
region_1        True
region_2        True
variety        False
winery         False
dtype: bool

In [6]:
wine.isnull().any().any()

True

Now, we need to deal with the null values in the wine table.
How we deal with them will depend on the data type of the column in which null values are found, and how we hope to use the data.

If a wine doesn't have a region listed, we can still get a lot of information about it from things like its country. But, if a wine doesn't have a price listed, it's difficult to get much information out of it. 

So, if a wine has a null value for either of its regions, we will set it to be "unknown". But, if it has a null price, we will drop the row.

In [8]:
wine.loc[wine.country.isnull()][:3]
# wine.loc[wine.country.isna(), 'country'] = 'Unknown' # set NaN in col country to 'Unknown'
# wine.loc[:, 'country'].fillna('Unknown', inplace=True)

,country,description,designation,points,price,province,region_1,region_2,variety,winery
1133,NaN,Delicate white flowers and a spin of lemon pee...,Askitikos,90,17.0,NaN,NaN,NaN,Assyrtiko,Tsililis
1440,NaN,"A blend of 60% Syrah, 30% Cabernet Sauvignon a...",Shah,90,30.0,NaN,NaN,NaN,Red Blend,Büyülübağ
68226,NaN,"From first sniff to last, the nose never makes...",Piedra Feliz,81,15.0,NaN,NaN,NaN,Pinot Noir,Chilcas


In [5]:
print('Shape of wine table before is: ', wine.shape)

null_country = wine.loc[wine['country'].isnull()]
print('Shape of null countries table is:', null_country.shape)

wine = wine.loc[~wine.index.isin(null_country.index)]
print('Shape of wine table after is: ', wine.shape)

Shape of wine table before is:  (150930, 10)
Shape of null countries table is: (5, 10)
Shape of wine table after is:  (150925, 10)


The 5 rows where a NaN value existed for country have been dropped.

Now we'll do the same for rows with a null price.

In [6]:
print('Shape of wine table before is: ', wine.shape)

null_price = wine.loc[wine['price'].isnull()]
print('Shape of null prices table is:', null_price.shape)

wine = wine[~wine.index.isin(null_price.index)]
print('Shape of wine table after is: ', wine.shape)

Shape of wine table before is:  (150925, 10)
Shape of null prices table is: (13695, 10)
Shape of wine table after is:  (137230, 10)


Wow! We lost a lot of rows getting rid of the null price rows.
Can you think of another way we could deal with null prices that doesn't mean losing almost 1/10 of our data? What are the pros and cons to using these other methods?

Now, for the other columns in which there are nulls, we can use `df.fillna()` to populate the null values with a value of our choosing. 

This is where we have to make some decisions. Improperly setting variables can have unintended consequences on any analysis we plan to do later using this data.

The only columns where we still have nulls are regions and designations.
To avoid having to drop out more rows, we will use "fillna()".

In [17]:
wine.fillna('unknown', inplace=True)

In [16]:
wine.isnull().any()

country         True
description    False
designation     True
points         False
price           True
province        True
region_1        True
region_2        True
variety        False
winery         False
Region          True
New_ID         False
dtype: bool

We now have no null values in our dataset!

In [18]:
wine.isnull().any()

country        False
description    False
designation    False
points         False
price          False
province       False
region_1       False
region_2       False
variety        False
winery         False
Region         False
New_ID         False
dtype: bool

In [19]:
# visualizing the df to see rows where "unknown" has been inserted
wine.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery,Region,New_ID
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,Napa Valley_Napa,US_Heitz_150
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,unknown,Tinta de Toro,Bodega Carmen Rodríguez,unknown,Spain_Bodega Carmen Rodríguez_330
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,Knights Valley_Sonoma,US_Macauley_279
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,Willamette Valley_Willamette Valley,US_Ponzi_786
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,unknown,Provence red blend,Domaine de la Bégude,unknown,France_Domaine de la Bégude_132


### Creating new columns from existing ones
In pandas, it's easy to make a new column from existing ones. 

In [20]:
wine.loc[:, 'Region'] = wine.loc[:, 'region_1'] + '_' + wine.loc[:, 'region_2']

#### Using a function + apply to make a new column:
Let's say we want to make a new column with an ID we can use to identify the records.
We can use a helper function + the apply method in pandas to apply it to all rows.
Apply looks through each row or column of a dataframe (depending on Axis set).

In [30]:
def add_unique_id(row):
    row['New_ID'] = (row['country'] + "_" + 
                     row['winery'] + "_" + 
                     str(np.random.randint(low=1, high=1000)))
    return row

In [31]:
wine = wine.apply(add_unique_id, axis=1)

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [25]:
wine.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery,Region,New_ID
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,Napa Valley_Napa,US_Heitz_33
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,unknown,Tinta de Toro,Bodega Carmen Rodríguez,Toro_unknown,Spain_Bodega Carmen Rodríguez_136
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,Knights Valley_Sonoma,US_Macauley_753
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,Willamette Valley_Willamette Valley,US_Ponzi_460
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,unknown,Provence red blend,Domaine de la Bégude,Bandol_unknown,France_Domaine de la Bégude_796


We can use the same approach to make a column for "great and cheap" wines, defined here as $50 or less and with a score >95.

In [32]:
def helper(row):
    if (row['price'] < 50) & (row['points'] >=95):
        x = 1
    else:
        x = 0
    return x

In [36]:
wine.price.fillna(0, inplace=True)
wine.price = wine.price.astype('int')

In [37]:
wine.loc[:, 'great and cheap'] = wine.apply(helper, axis=1)

We can then use that boolean value to sort on the wines.

In [47]:
cheap_great = wine.loc[wine.loc[:, 'great and cheap']==1]

cheap_great[cheap_great.country=='Germany']\
    .loc[:, ~cheap_great.columns.isin(['index', 'region_1', 'region_2'])]\
    .sort_values(by=['price', 'points'], ascending=[True, False])

,country,description,designation,points,price,province,variety,winery,great and cheap
6014,Germany,Keller's Grosses Gewächs offerings are hard to...,von der Fels,95,0,Rheinhessen,Riesling,Weingut Keller,1
127776,Germany,One of the most remarkable things about the J....,Wehlener Sonnenuhr Auslese Goldkap,95,0,Mosel-Saar-Ruwer,Riesling,Joh. Jos. Prüm,1
10875,Germany,"This intensely fruity, floral nectar hits the ...",Ürziger Würzgarten Spätlese,95,34,Mosel,Riesling,Dr. Loosen,1
10878,Germany,"Tasted five years after bottling, this profoun...",Wehlener Sonnenuhr Spätlese Grosse Lage,95,35,Mosel,Riesling,S.A. Prüm,1
26672,Germany,The scents of dried apricot and honey are made...,Jungfer Auslese,95,44,Rheingau,Riesling,Prinz,1
127498,Germany,Bottling a TBA in this format makes great sens...,Wehlener Sonnenuhr Trockenbeerenauslese Goldkap,96,49,Mosel-Saar-Ruwer,Riesling,Dr. Loosen,1


**Exercise:** Make a new column in the energy dataset, showing the energy in time T-1 for each time T. Hint: there is a built-in method in Pandas that can help with this, no complicated functions needed :)

In [19]:
energy.loc[:, 'energy_t-1'] = energy.loc[:, 'PJM_Load_MW'].shift()

In [20]:
energy.head()

,PJM_Load_MW,energy_t-1
Datetime,,
1998-12-31 01:00:00,29309.0,NaN
1998-12-31 02:00:00,28236.0,29309.0
1998-12-31 03:00:00,27692.0,28236.0
1998-12-31 04:00:00,27596.0,27692.0
1998-12-31 05:00:00,27888.0,27596.0


**Exercise:** Re-read the CSV for the wine data, and write a function to take care of all the nan values. Bonus points if it includes a test.

In [49]:
wine1 = pd.read_csv('data/wine_reviews/winemag-data_first150k.csv', index_col=0)

def fix_null_values(wine_df, cols_to_drop, cols_to_fill):
    """
    Function to remove null values in the wine dataframe.
    Args:
        wine_df (DataFrame): DataFrame from which to remove the null values.
        cols_to_drop (list): List of column names to drop rows from
        cols_to_fill (list): List of columns names to fill with "unknown"
    Returns:
        wine_df (DataFrame): DataFrame with null values removed
    """
    for col in cols_to_drop:
        indicies_to_drop = wine_df.loc[wine_df[col].isnull()] # get indicies from rows where price or country is null
        wine_df = wine_df[~wine_df.index.isin(indicies_to_drop.index)]
        
    for col in cols_to_fill:
        wine_df[col].fillna('unknown', inplace=True) 
        
    # checking that all null values have been removed
    if wine_df.isnull().any().any() == False:
        return wine_df
    
    # if there are still null values, returning column name to user
    else:
        all_cols_list = list(wine_df.isnull().any())
        list_cols_with_nulls = [col for i, col in enumerate(all_cols_list.index) if all_cols_list[i] is True]
        print(f"Null values still exist! Columns {list_cols_with_nulls}")
        return
        
to_drop = ["price", "country"]
to_fill = ['description', 'designation', 'points', 'province', 'region_1', 'region_2', 'variety', 'winery']

# calling funciton
fix_null_values(wine1, to_drop, to_fill)

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,unknown,Tinta de Toro,Bodega Carmen Rodríguez
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,unknown,Provence red blend,Domaine de la Bégude
...,...,...,...,...,...,...,...,...,...,...
150925,Italy,Many people feel Fiano represents southern Ita...,unknown,91,20.0,Southern Italy,Fiano di Avellino,unknown,White Blend,Feudi di San Gregorio
150926,France,"Offers an intriguing nose with ginger, lime an...",Cuvée Prestige,91,27.0,Champagne,Champagne,unknown,Champagne Blend,H.Germain
150927,Italy,This classic example comes from a cru vineyard...,Terre di Dora,91,20.0,Southern Italy,Fiano di Avellino,unknown,White Blend,Terredora
150928,France,"A perfect salmon shade, with scents of peaches...",Grand Brut Rosé,90,52.0,Champagne,Champagne,unknown,Champagne Blend,Gosset


## Updates: Pandas 1.0
Pandas had already been a well-tested package used in development environments for quite some time. The official move to version 1.0 mainly just included some format fixes and 
- Dedicated types for String, boolean and NaN (before, just called "object")
- New documentation


In [50]:
!pip install tabulate

In [51]:
wine.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery,great and cheap
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,0
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez,0
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,0
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,0
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude,0


In [52]:
# Convert columns to the best possible dtypes using dtypes supporting
wine.convert_dtypes()

,country,description,designation,points,price,province,region_1,region_2,variety,winery,great and cheap
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,0
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110,Northern Spain,Toro,<NA>,Tinta de Toro,Bodega Carmen Rodríguez,0
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,0
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,0
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66,Provence,Bandol,<NA>,Provence red blend,Domaine de la Bégude,0
...,...,...,...,...,...,...,...,...,...,...,...
150925,Italy,Many people feel Fiano represents southern Ita...,<NA>,91,20,Southern Italy,Fiano di Avellino,<NA>,White Blend,Feudi di San Gregorio,0
150926,France,"Offers an intriguing nose with ginger, lime an...",Cuvée Prestige,91,27,Champagne,Champagne,<NA>,Champagne Blend,H.Germain,0
150927,Italy,This classic example comes from a cru vineyard...,Terre di Dora,91,20,Southern Italy,Fiano di Avellino,<NA>,White Blend,Terredora,0
150928,France,"A perfect salmon shade, with scents of peaches...",Grand Brut Rosé,90,52,Champagne,Champagne,<NA>,Champagne Blend,Gosset,0


In [54]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150930 entries, 0 to 150929
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   country          150925 non-null  object
 1   description      150930 non-null  object
 2   designation      105195 non-null  object
 3   points           150930 non-null  int64 
 4   price            150930 non-null  int64 
 5   province         150925 non-null  object
 6   region_1         125870 non-null  object
 7   region_2         60953 non-null   object
 8   variety          150930 non-null  object
 9   winery           150930 non-null  object
 10  great and cheap  150930 non-null  int64 
dtypes: int64(3), object(8)
memory usage: 13.8+ MB
